![Alt Text]("https://www.heartoflongmont.org/wp-content/uploads/2019/02/Movie-Recommendation.jpg")

<div style="text-align:center">
    <img src="https://www.heartoflongmont.org/wp-content/uploads/2019/02/Movie-Recommendation.jpg" alt="movie  Text">
   
</div>

# Tutorial: Designing a Recommendation System
> developed by rimmel asghar

In [1]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/imdb-movies/movies.csv


In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
import re

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
df = pd.read_csv('/kaggle/input/imdb-movies/movies.csv')
df.head()

,poster,title,certificate,runtime,genre,rating,about,director,stars,votes,gross_earn
0,https://m.media-amazon.com/images/S/sash/4Fyxw...,The Shawshank Redemption,15,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,Frank Darabont,"('Tim Robbins',), ('Morgan Freeman',), ('Bob G...","26,26,905",$28.34M
1,https://m.media-amazon.com/images/S/sash/4Fyxw...,The Dark Knight,12A,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,Christopher Nolan,"('Christian Bale',), ('Heath Ledger',), ('Aaro...","25,98,173",$534.86M
2,https://m.media-amazon.com/images/S/sash/4Fyxw...,Inception,12A,148 min,"Action, Adventure, Sci-Fi",8.8,A thief who steals corporate secrets through t...,Christopher Nolan,"('Leonardo DiCaprio',), ('Joseph Gordon-Levitt...","23,04,062",$292.58M
3,https://m.media-amazon.com/images/S/sash/4Fyxw...,Fight Club,18,139 min,Drama,8.8,An insomniac office worker and a devil-may-car...,David Fincher,"('Brad Pitt',), ('Edward Norton',), ('Meat Loa...","20,71,088",$37.03M
4,https://m.media-amazon.com/images/S/sash/4Fyxw...,Pulp Fiction,18,154 min,"Crime, Drama",8.9,"The lives of two mob hitmen, a boxer, a gangst...",Quentin Tarantino,"('John Travolta',), ('Uma Thurman',), ('Samuel...","20,11,013",$107.93M


In [4]:
df.columns

Index(['poster', 'title', 'certificate', 'runtime', 'genre', 'rating', 'about',
       'director', 'stars', 'votes', 'gross_earn'],
      dtype='object')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   poster       5000 non-null   object 
 1   title        5000 non-null   object 
 2   certificate  5000 non-null   object 
 3   runtime      5000 non-null   object 
 4   genre        4970 non-null   object 
 5   rating       5000 non-null   float64
 6   about        4986 non-null   object 
 7   director     5000 non-null   object 
 8   stars        5000 non-null   object 
 9   votes        5000 non-null   object 
 10  gross_earn   4585 non-null   object 
dtypes: float64(1), object(10)
memory usage: 429.8+ KB


In [6]:
df.describe()

,rating
count,5000.000000
mean,6.692300
std,0.973649
min,1.500000
25%,6.100000
50%,6.800000
75%,7.400000
max,9.300000


In [7]:
df.isnull().sum()

poster           0
title            0
certificate      0
runtime          0
genre           30
rating           0
about           14
director         0
stars            0
votes            0
gross_earn     415
dtype: int64

In [8]:
new_df = df[['title', 'certificate', 'genre', 'about', 'director', 'stars']]
new_df.head()

,title,certificate,genre,about,director,stars
0,The Shawshank Redemption,15,Drama,Two imprisoned men bond over a number of years...,Frank Darabont,"('Tim Robbins',), ('Morgan Freeman',), ('Bob G..."
1,The Dark Knight,12A,"Action, Crime, Drama",When the menace known as the Joker wreaks havo...,Christopher Nolan,"('Christian Bale',), ('Heath Ledger',), ('Aaro..."
2,Inception,12A,"Action, Adventure, Sci-Fi",A thief who steals corporate secrets through t...,Christopher Nolan,"('Leonardo DiCaprio',), ('Joseph Gordon-Levitt..."
3,Fight Club,18,Drama,An insomniac office worker and a devil-may-car...,David Fincher,"('Brad Pitt',), ('Edward Norton',), ('Meat Loa..."
4,Pulp Fiction,18,"Crime, Drama","The lives of two mob hitmen, a boxer, a gangst...",Quentin Tarantino,"('John Travolta',), ('Uma Thurman',), ('Samuel..."


In [9]:
new_df.isnull().sum()

title           0
certificate     0
genre          30
about          14
director        0
stars           0
dtype: int64

In [10]:
new_df = new_df.dropna().reset_index()

In [11]:
new_df.columns

Index(['index', 'title', 'certificate', 'genre', 'about', 'director', 'stars'], dtype='object')

In [12]:
new_df.head()

,index,title,certificate,genre,about,director,stars
0,0,The Shawshank Redemption,15,Drama,Two imprisoned men bond over a number of years...,Frank Darabont,"('Tim Robbins',), ('Morgan Freeman',), ('Bob G..."
1,1,The Dark Knight,12A,"Action, Crime, Drama",When the menace known as the Joker wreaks havo...,Christopher Nolan,"('Christian Bale',), ('Heath Ledger',), ('Aaro..."
2,2,Inception,12A,"Action, Adventure, Sci-Fi",A thief who steals corporate secrets through t...,Christopher Nolan,"('Leonardo DiCaprio',), ('Joseph Gordon-Levitt..."
3,3,Fight Club,18,Drama,An insomniac office worker and a devil-may-car...,David Fincher,"('Brad Pitt',), ('Edward Norton',), ('Meat Loa..."
4,4,Pulp Fiction,18,"Crime, Drama","The lives of two mob hitmen, a boxer, a gangst...",Quentin Tarantino,"('John Travolta',), ('Uma Thurman',), ('Samuel..."


## Preprocessing

Steps:

-  We will use genre, about, director, stars in single column as tags.

- Remove Stop words

- convert tags into Vectors.

In [13]:
new_df['stars']  = new_df['stars'].apply(lambda x : re.sub(r"[()']", '', x))
new_df.head()

,index,title,certificate,genre,about,director,stars
0,0,The Shawshank Redemption,15,Drama,Two imprisoned men bond over a number of years...,Frank Darabont,"Tim Robbins,, Morgan Freeman,, Bob Gunton,, Wi..."
1,1,The Dark Knight,12A,"Action, Crime, Drama",When the menace known as the Joker wreaks havo...,Christopher Nolan,"Christian Bale,, Heath Ledger,, Aaron Eckhart,..."
2,2,Inception,12A,"Action, Adventure, Sci-Fi",A thief who steals corporate secrets through t...,Christopher Nolan,"Leonardo DiCaprio,, Joseph Gordon-Levitt,, Ell..."
3,3,Fight Club,18,Drama,An insomniac office worker and a devil-may-car...,David Fincher,"Brad Pitt,, Edward Norton,, Meat Loaf,, Zach G..."
4,4,Pulp Fiction,18,"Crime, Drama","The lives of two mob hitmen, a boxer, a gangst...",Quentin Tarantino,"John Travolta,, Uma Thurman,, Samuel L. Jackso..."


('Tim Robbins',), will turn into this Tim Robbins,,

In [14]:
new_df['director'] = new_df['director'].str.replace(' ', '')
new_df['stars'] = new_df['stars'].str.replace(' ', '')
new_df.head()

,index,title,certificate,genre,about,director,stars
0,0,The Shawshank Redemption,15,Drama,Two imprisoned men bond over a number of years...,FrankDarabont,"TimRobbins,,MorganFreeman,,BobGunton,,WilliamS..."
1,1,The Dark Knight,12A,"Action, Crime, Drama",When the menace known as the Joker wreaks havo...,ChristopherNolan,"ChristianBale,,HeathLedger,,AaronEckhart,,Mich..."
2,2,Inception,12A,"Action, Adventure, Sci-Fi",A thief who steals corporate secrets through t...,ChristopherNolan,"LeonardoDiCaprio,,JosephGordon-Levitt,,ElliotP..."
3,3,Fight Club,18,Drama,An insomniac office worker and a devil-may-car...,DavidFincher,"BradPitt,,EdwardNorton,,MeatLoaf,,ZachGrenier,"
4,4,Pulp Fiction,18,"Crime, Drama","The lives of two mob hitmen, a boxer, a gangst...",QuentinTarantino,"JohnTravolta,,UmaThurman,,SamuelL.Jackson,,Bru..."


In [15]:
new_df['genre'] = new_df['genre'].apply(lambda x : x.split())
new_df['certificate'] = new_df['certificate'].apply(lambda x : x.split())
new_df['about'] = new_df['about'].apply(lambda x : x.split())
new_df['director'] = new_df['director'].apply(lambda x : x.split())
new_df['stars'] = new_df['stars'].apply(lambda x : x.split())
new_df.head(2)

,index,title,certificate,genre,about,director,stars
0,0,The Shawshank Redemption,[15],[Drama],"[Two, imprisoned, men, bond, over, a, number, ...",[FrankDarabont],"[TimRobbins,,MorganFreeman,,BobGunton,,William..."
1,1,The Dark Knight,[12A],"[Action,, Crime,, Drama]","[When, the, menace, known, as, the, Joker, wre...",[ChristopherNolan],"[ChristianBale,,HeathLedger,,AaronEckhart,,Mic..."


In [16]:
new_df['Tags'] = new_df['certificate'] + new_df['genre'] + new_df['about'] + new_df['director'] + new_df['stars']
new_df.head(3)

,index,title,certificate,genre,about,director,stars,Tags
0,0,The Shawshank Redemption,[15],[Drama],"[Two, imprisoned, men, bond, over, a, number, ...",[FrankDarabont],"[TimRobbins,,MorganFreeman,,BobGunton,,William...","[15, Drama, Two, imprisoned, men, bond, over, ..."
1,1,The Dark Knight,[12A],"[Action,, Crime,, Drama]","[When, the, menace, known, as, the, Joker, wre...",[ChristopherNolan],"[ChristianBale,,HeathLedger,,AaronEckhart,,Mic...","[12A, Action,, Crime,, Drama, When, the, menac..."
2,2,Inception,[12A],"[Action,, Adventure,, Sci-Fi]","[A, thief, who, steals, corporate, secrets, th...",[ChristopherNolan],"[LeonardoDiCaprio,,JosephGordon-Levitt,,Elliot...","[12A, Action,, Adventure,, Sci-Fi, A, thief, w..."


In [17]:
final_df = new_df[['title', 'Tags']].copy()  # Create a copy to avoid modifying the original DataFrame
final_df['Tags'] = final_df['Tags'].apply(lambda x: ' '.join(x))
final_df.head()

,title,Tags
0,The Shawshank Redemption,15 Drama Two imprisoned men bond over a number...
1,The Dark Knight,"12A Action, Crime, Drama When the menace known..."
2,Inception,"12A Action, Adventure, Sci-Fi A thief who stea..."
3,Fight Club,18 Drama An insomniac office worker and a devi...
4,Pulp Fiction,"18 Crime, Drama The lives of two mob hitmen, a..."


## Stemming
Stemming is a text preprocessing technique used in natural language processing and information retrieval to reduce words to their base or root form. The process involves removing suffixes or prefixes from words to obtain a common form that represents various grammatical variations of a word.

For example, consider the words "running," "runner," and "runs." By applying stemming, these words would be reduced to their common root form, which is "run."

In [18]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return ' '.join(y)
final_df['Tags'] = final_df['Tags'].apply(stem)

In [19]:
final_df['Tags'] = final_df['Tags'].apply(lambda x : x.lower())

## CountVectorizer
A CountVectorizer is a text preprocessing technique used in natural language processing (NLP) to convert a collection of text documents into a matrix of token counts. It's a common step in preparing text data for machine learning algorithms, particularly those that require numerical input.

In [20]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(max_features=10000, stop_words='english')
vectors = cv.fit_transform(final_df['Tags']).toarray()
feature_names = cv.get_feature_names_out()

print(feature_names)

['00' '000' '007' ... 'émilien' 'ömerfaruksorak' 'özgeözberk']


## Cosine similarity
Cosine similarity is a measure used to determine how similar two documents or vectors are based on their orientation in a multi-dimensional space. It's commonly used in recommendation systems to find similar items based on user preferences or content features

## Recommendation By Cosine Similarity

In [21]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vectors)

In [22]:
def get_recomm(movie):
    movie_index  = final_df[final_df['title'] == movie].index[0]
    recom = sorted(list(enumerate(similarity[movie_index])), reverse = True, key = lambda x : x[1])[1 : 6]
    for i in recom:
        print(final_df.iloc[i[0]].title)

In [28]:
get_recomm('Fight Club')

Crash
Seven
Blood and Bone
Tyrannosaur
A.W.O.L.: Absent Without Leave


**Exporting the model file and the dataframe in a pickle file format.**

In [29]:
import pickle
pickle.dump(final_df.to_dict(), open('IMDB-0.1.0.pkl', 'wb'))

In [31]:
pickle.dump(similarity, open('model-0.1.0.pkl', 'wb'))     